<a href="https://colab.research.google.com/github/jvishnuvardhan/Stackoverflow_Questions/blob/master/Save_load_model_with_customeObjects_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly

     |████████████████████████████████| 517.1MB 28kB/s 
     |████████████████████████████████| 460kB 49.1MB/s 
     |████████████████████████████████| 2.9MB 42.7MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Custom Loss (for example) 
@tf.function() 
def triplet_loss(y_true, y_pred, alpha = 0.3):
  anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
  pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
  neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
  basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
  loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
  return loss
  
def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),  
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

  model.compile(optimizer='adam', loss=triplet_loss)
  return model

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
# Create a basic model instance
model=create_model()

# Fit and evaluate model 
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3004
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3002
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3000
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3000
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3000


In [0]:
# save the model in *.h5 format
model.save('./MyModel.h5',save_format='h5')

In [0]:
# load the saved model (Don't compile while loading as custom objects are not serialized)
loaded_model = tf.keras.models.load_model('./MyModel.h5', custom_objects={triplet_loss:triplet_loss},compile=False)

In [0]:
# compile the loaded model with the custome objects
loaded_model.compile(optimizer='adam',loss=triplet_loss)